In [1]:
from allocation.Enums import FdApproach
from allocation.eulerAllocator import EulerAllocator
from collateralAgreement.collateralAgreement import CollateralAgreement
from instruments.interestRateInstrument.irs import IRS
from marketdata.interestRateIndices import InterestRateIndex
from sa_ccr.sa_ccr import SA_CCR
import QuantLib as ql
from marketdata import init_marketdata

In [2]:
pf1 = CollateralAgreement()
pf1.link_sa_ccr_instance(SA_CCR(pf1))
pf1_bumped = CollateralAgreement()
pf1_bumped.link_sa_ccr_instance(SA_CCR(pf1_bumped))
pf1_bumped_rel = CollateralAgreement()
pf1_bumped_rel.link_sa_ccr_instance(SA_CCR(pf1_bumped_rel))


irs1 = IRS(notional = 20000000,
           timeToSwapStart=ql.Period(2, ql.Days),
           timeToSwapEnd=ql.Period(10, ql.Years),
           index = InterestRateIndex.USDLIBOR3M)

irs2 = IRS(notional = 400000000000,
           timeToSwapStart=ql.Period(2, ql.Days),
           timeToSwapEnd=ql.Period(10, ql.Years),
           index = InterestRateIndex.USDLIBOR3M)

irs1_abs_bumped = irs1.get_bumped_copy(rel_bump_size=0.000001, bump_approach=FdApproach.Relative)
irs2_abs_bumped = irs2.get_bumped_copy(rel_bump_size=0.000001, bump_approach=FdApproach.Relative)

# irs1_rel_bumped = irs1.get_bumped_copy()

In [3]:
pf1.add_trades(irs1)
irs_1_isda_simm = pf1.get_im_model().get_risk_measure()

pf1_bumped.add_trades(irs1_abs_bumped)
irs_1_bumped_isda_simm = pf1_bumped.get_im_model().get_risk_measure()

euler_alloc_manuel_irs_1 = (irs_1_bumped_isda_simm-irs_1_isda_simm)/0.000001

euler_alloc_manuel_irs_2 = ()

# pf1_bumped_rel.add


eulerAllocator_1 = EulerAllocator(pf1)
allocated_irs_1 = eulerAllocator_1.allocate_im()